In [1]:

import pandas as pd
import json
from pkg_resources import resource_filename
import numpy as np

In [3]:
tounament=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WNCAATourneyCompactResults.csv')
seed=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WNCAATourneySeeds.csv')
rounds=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WNCAATourneySeedRoundSlots.csv')
slots=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WNCAATourneySlots.csv')
teams=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WTeams.csv')
regions=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/WSeasons.csv')
teams=teams[['TeamID','TeamName']]

In [4]:
r1=rounds.rename(columns={'EarlyDayNum':'Daynum'})
r2=rounds.rename(columns={'LateDayNum':'Daynum'})
double_rounds=pd.concat([r1,r2],axis=0)
double_rounds

NameError: name 'rounds' is not defined

In [ ]:
display(seed),display(rounds),display(tounament),display(teams)

In [ ]:
#change the day num to round
round_map={134:0,
          135:0,

          136:1,
          137:1,
        #   140:1,

          138:2,
          139:2,

          143:3,
          144:3,

          145:4,
          146:4,          
        #   147:4,
        #   148:4,            

          152:5,
          
          154:6}

In [ ]:
#Standardize the Regions
region_map={'South':'South',
 'MidWest':'Midwest',
 'East':'East',
 'West':'West',
 
 'Southeast':'South',
 'Southwest':'Midwest',

 'Albuquerque':'West',
 'Chicago':'Midwest',
 'Austin':'South',
 'Syracuse':'East',

 'NA1':'East',
 'NA2':'Midwest',
 'NA4':'West',
 'NA3':'South',

 'Oakland':'West',
 'Atlanta':'South',
 'WashingtonDC':'East',
 'Minneapolis':'Midwest',
 'EastRutherford':'East',

 'StLouis':'Midwest',
 'Atlanta':'South',
 'Phoenix':'West',
}

In [ ]:
region_to_num={'W':1,
               'X':2,
               'Y':3,
               'Z':4}


In [ ]:
regions.filter(like='Region').columns
all_regions=[]

regions.columns=[c.replace('Region','') for c in regions.columns]
seed_region_map=regions.set_index('Season')[['W','X','Y','Z']].T.to_dict()
for season in seed_region_map:
    seed_region_map[season]={i:a for a,i in seed_region_map[season].items()}


In [ ]:
seeds=pd.merge(seed,teams,left_on='TeamID',right_on='TeamID',how='left')

seeds['Region']=seeds['Seed'].str[0]


In [ ]:
region_to_region={'W':'East',
               'X':'Midwest',
               'Y':'South',
               'Z':'West'}

In [ ]:
seasons=seeds.Season.unique()
for season in seasons:
    seeds.loc[seeds.Season==season,'Region_']=seeds.loc[seeds.Season==season,'Region'].replace(seed_region_map[season])
seeds['RegionNum']=seeds['Region'].replace(region_to_num)


seed_df=seeds
seed_df

In [ ]:
kaggle_names=sorted(teams['TeamName'].to_list())


In [ ]:
tournament_df=pd.merge(tounament,teams,left_on=['WTeamID'],right_on=['TeamID'],how='left')
tournament_df=tournament_df.rename(columns={'TeamName':'WTeam'})
tournament_df=tournament_df.drop('TeamID',axis=1)
########
tournament_df=pd.merge(tournament_df,teams,left_on=['LTeamID'],right_on=['TeamID'],how='left')
tournament_df=tournament_df.rename(columns={'TeamName':'LTeam'})
tournament_df['Round']=tournament_df['DayNum'].replace(round_map)
########
tournament_df=tournament_df.drop(['TeamID'],axis=1)
tournament_df.columns=[C.title() for C in tournament_df.columns]


In [ ]:
#change the day num to round
round_map_2021={
140:3}
tournament_df.loc[tournament_df.Season==2021,'Round']=tournament_df.loc[tournament_df.Season==2021,'Daynum'].replace(round_map_2021)

In [ ]:
## merge Seeds with the teams
tourn_region=pd.merge(tournament_df,seeds[['TeamName','Seed','Season','Region','RegionNum']],
left_on=['Season','Wteam'],
right_on=['Season','TeamName'],
how='left'
)
tourn_region=tourn_region.drop(['TeamName','Wloc','Numot'],axis=1)
tourn_region=tourn_region.rename(columns={'Seed':'Wseed'})


tourn_region=pd.merge(tourn_region,seeds[['Season','TeamName','Seed']],
left_on=['Season','Lteam'],
right_on=['Season','TeamName'],
how='left'
)

tourn_region=tourn_region.rename(columns={'Seed':'Lseed'})
tourn_region=tourn_region.drop('TeamName',axis=1)


In [ ]:
r=1
print(tourn_region[(tourn_region.Season==2021)].shape[0])

tourn2021=tourn_region[(tourn_region.Season==2021)]
add_game=tourn2021.loc[[tourn2021.index[0]]].copy()

add_game.Season=2021
add_game.Daynum=138
add_game.Wteam='Oregon'
add_game.Wseed='X07'
add_game.Wscore=1

add_game.Lteam='VCU'
add_game.Lseed='X10'
add_game.Lscore=0

add_game.Region='X'
add_game.RegionNum=2
add_game.Round=1

tourn_region=pd.concat([tourn_region,add_game],axis=0,ignore_index=True)
tourn_region,add_game

In [ ]:
## there was a no contest in 2022 so we have to add vcu and Oregon into the teams
tourn2021=tourn_region[(tourn_region.Season==2021)]
tourn2021=tourn_region[(tourn_region.Season==2021)]
for i in tourn2021.Round.unique():
    print(i)
    display(tourn2021[tourn2021.Round==i].shape)


In [ ]:

tourn_region['RegionNum']=tourn_region['RegionNum'].astype(int)
Wseed=(tourn_region['Wseed'].str.replace("[a-zA-Z]","",regex=True)).astype(int)
Lseed=(tourn_region['Lseed'].str.replace("[a-zA-Z]","",regex=True)).astype(int)
real_seeds=tourn_region[['Wseed','Lseed']].values
choose_seeds=np.concatenate([Wseed.values.reshape(-1,1),Lseed.values.reshape(-1,1)],axis=1)


In [ ]:
teams=tourn_region[['Wteam','Lteam']].values


for i in range(len(choose_seeds)):
    g=choose_seeds[i,:]
    if g[0]==g[1]:
        choose_seeds[i,0]+=1
        
high_arg=np.argmax(choose_seeds,1).reshape(-1,1)
low_arg=np.argmin(choose_seeds,1).reshape(-1,1)
tourn_region['high_team']=np.take_along_axis(teams,high_arg,1)
tourn_region['high_seed']=np.take_along_axis(real_seeds,high_arg,1)
tourn_region['low_team']=np.take_along_axis(teams,low_arg,1)
tourn_region['low_seed']=np.take_along_axis(real_seeds,low_arg,1)
tourn_region=tourn_region.rename(columns={'Season':'Year'})
# tourn_region['Region']=tourn_region['Region'].replace(region_to_region)


In [ ]:
tourn_region

In [ ]:
base_columns = ['Year','Round','Daynum','Region','RegionNum']
team0_columns = ['high_team','high_seed']
team1_columns = ['low_team','low_seed']
rename_columns=['Team','Seed']

In [ ]:
high_seed_df = tourn_region[base_columns + team0_columns].rename(columns=dict(zip(team0_columns,rename_columns))).copy()
low_seed_df  = tourn_region[base_columns + team1_columns].rename(columns=dict(zip(team1_columns,rename_columns))).copy()

high_seed_df['Team_Position'] = 'high'
low_seed_df['Team_Position'] = 'low'
raw_long_df = pd.concat([high_seed_df, low_seed_df],axis=0)
raw_long_df ['Mapped_Round']=raw_long_df ['Round']
raw_long_df=raw_long_df.drop('Round',axis=1)
display(double_rounds)
raw_long_df


In [ ]:
tourn_with_rounds=pd.merge(raw_long_df,double_rounds,left_on=['Daynum','Seed'],right_on=['Daynum','Seed'])
tourn_with_rounds.Seed.isna().sum()

In [ ]:
new_year=seed_df[seed_df.Season==2023]
new_year=new_year.rename(columns={'TeamName':'Team','Season':'Year'})
new_year['Seed']=new_year['Seed']#.str.replace("[a-zA-Z]","",regex=True).astype(int)
new_year['GameRound']=1


In [ ]:
long_tourn =tourn_with_rounds.reset_index().copy()
y2021idx=long_tourn[(long_tourn.Year==2019)].index
# long_tourn .loc[y2021idx,'GameRound']=long_tourn .loc[y2021idx,'Mapped_Round']

y2022=long_tourn .loc[y2021idx,['Daynum','GameRound']]
y2022.value_counts().sort_index(),y2022.shape,y2022.value_counts().sum()
# long_tourn.loc[y2021idx]

In [ ]:


long_tourn =tourn_with_rounds.reset_index().copy()
y2021idx=long_tourn[(long_tourn.Year==2022)].index
# long_tourn .loc[y2021idx,'GameRound']=long_tourn .loc[y2021idx,'Mapped_Round']

y2022=long_tourn .loc[y2021idx,['Daynum','GameRound']]
y2022.value_counts().sort_index(),y2022.shape,y2022.value_counts().sum()
# long_tourn.loc[y2021idx]

In [ ]:

long_tourn =tourn_with_rounds.reset_index().copy()
y2021idx=long_tourn[(long_tourn.Year==2021)].index
# long_tourn .loc[y2021idx,'GameRound']=long_tourn .loc[y2021idx,'Mapped_Round']

y2021=long_tourn .loc[y2021idx,['Daynum','GameRound']]
y2021.value_counts().sort_index(),y2021.shape,y2021.value_counts().sum()

In [ ]:
#change the day num to round
round_map_2021={
         ## first 4
          136:0,## first 4

          ## first round
          137:1,
          138:1,

          # # second round

          139:2,
          
          # # sweet 16
          145:3,
          146:3,

          # # elite 8        
          147:4,
          148:4,            
          # # final 4  
          152:5,
          # # chamionship 
          154:6}

In [ ]:
last_year=long_tourn[long_tourn.Year==2022].copy()
last_year

last_year.loc[last_year.index,'Team']=['TBD' for c in range(len(last_year))]
last_year.loc[last_year.index,'Seed']=['64' for c in range(len(last_year))]
last_year.loc[last_year.index,'Year']=[2023 for c in range(len(last_year))]


last_year=last_year[last_year.GameRound>1]
last_year
long_tourn=pd.concat([long_tourn,new_year,last_year],axis=0,ignore_index=True)
long_tourn=long_tourn.rename(columns={'GameRound':'Round'})

long_tourn['Seed']=(long_tourn['Seed'].str.replace("[a-zA-Z]","",regex=True)).astype(int)
tourn_region['Seed']=(tourn_region['Wseed'].str.replace("[a-zA-Z]","",regex=True)).astype(int)
long_tourn=long_tourn.drop(
  ['LateDayNum','EarlyDayNum','TeamID','Region_'],axis=1
).sort_values(['Year','Round','Seed'])
long_tourn

In [ ]:
final_four_games = \
long_tourn[long_tourn['Round'] == 4] \
    .assign(game_num = lambda x: (x['RegionNum'] <= 2)+1) \
    .groupby(['Year','game_num']) \
    .agg({'Region' : lambda x: dict(zip(['team1','team2'],x))}) \
    .groupby('Year') \
    .agg({'Region' : lambda x: dict(zip(['game1','game2'],x))}) \
    .to_dict().get('Region')

In [ ]:
years = long_tourn.Year.unique().tolist()

brackets_dict = {}
for year in years:

    that_year_df = long_tourn[long_tourn['Year'] == year].copy()
    that_year_win = tourn_region[tourn_region['Year'] == year].copy()
    
    round0= that_year_df[that_year_df['Round'] == 0].sort_values('Seed')

    round1 = that_year_df[that_year_df['Round'] == 1].sort_values('Seed')
    round2 = that_year_df[that_year_df['Round'] == 2]
    round3 = that_year_df[that_year_df['Round'] == 3]
    round4 = that_year_df[that_year_df['Round'] == 4]
    round5 = that_year_df[that_year_df['Round'] == 5]
    round6 = that_year_df[that_year_df['Round'] == 6]

    print(that_year_df['Round'].unique())
    round6_win = that_year_win[that_year_win['Round'] == 6]
    round6_win=round6_win.rename(columns={'Wteam':'Team','Wseed':'Seed'})

    try:
        winner = round6_win[['Team','Seed']].reset_index(drop=True).T.to_dict()[0]

    except:
        
        
        winner={'Team': 'TBD', 'Seed': 0}
    # print(winner)
    
    east1 = round1[round1['Region'] == 'W']
    west1 = round1[round1['Region'] == 'X']
    midwest1 = round1[round1['Region'] == 'Y']
    south1 = round1[round1['Region'] == 'Z']

    regs={
            'W':east1,
            'X':west1,
            'Y':midwest1,
            'Z':south1,
                }

    print(year,len(round1))
    if i !=2021:
        for r in regs:
            if len(regs[r])!=16:
                print(round1['Region'].unique())
                display(r,regs[r])
                regs[r]=regs[r].drop_duplicates('Seed',keep='first').reset_index(drop=True)
                display(r,regs[r])



    results = {
        # 'playin' : round0[['Team','Seed']].to_dict('records'),
        'first' : round1[['Team','Seed']].to_dict('records'),
        'second' : round2[['Team','Seed']].to_dict('records'),
        'sweet16' : round3[['Team','Seed']].to_dict('records'),
        'elite8' : round4[['Team','Seed']].to_dict('records'),
        'final4' : round5[['Team','Seed']].to_dict('records'),
        'championship' : round6[['Team','Seed']].to_dict('records'),
        'winner' : {'Team' : winner['Team'], 'Seed' : int(winner['Seed'])}
    }
    regions = {
        'W' : regs['W'][['Team','Seed']].to_dict('records'),
        'X' : regs['X'][['Team','Seed']].to_dict('records'),
        'Y' : regs['Y'][['Team','Seed']].to_dict('records'),
        'Z' : regs['Z'][['Team','Seed']].to_dict('records')
    }
    brackets_dict[year] = {'Region' : regions, 'Results': results, 'Finals': final_four_games[year]}

In [ ]:
import json
with open('/Users/sebastiancoombs/IDrive-Sync/MetaLocalLLC/RL-Bots/Tourney_gym/bracketology_rl/data/Womens_brackets_2022.json' , 'w') as f:
    json.dump(brackets_dict, f)